In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from __future__ import absolute_import, division, print_function

import pathlib

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, explained_variance_score
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers import LSTM
from datetime import timedelta
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import QuantileTransformer

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

print(tf.__version__)

In [ ]:
# Use seaborn for pairplot
!pip install -q seaborn

Import it using pandas

In [ ]:
column_names = ['Location', 'Year', 'Month', 'Day', 'Hour', 'Cloud_Cover_Fraction',
                'Dew_Point','Humidity_Fraction','Precipitation',
                'Pressure', 'Temperature', 'Visibility', 'Wind_Speed',
                'Solar_Elevation', 'Electricity_KW_HR'] 
#raw_dataset = pd.read_csv("../input/df_solararray_complete.csv",
#                      na_values = "NA", comment='\t',
#                      sep=",")
raw_dataset = pd.read_csv("../input/df_solararray_complete.csv")
# , names=column_names, skipinitialspace=True
dataset = raw_dataset.copy()
dataset.head()

In [ ]:
df=dataset.drop(['Unnamed: 0', 'Location'], axis=1)
df.head()
# hour must be 0 - 23
df['Hour'] = df['Hour'] - 1

# create datetime column
df['datetime'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']],infer_datetime_format=True)

df = df.set_index(['datetime'],drop=True)
df.head()

In [ ]:
# may have to do something about why these values for precipitation 
# and Pressure missing. For now we will remove it since its only 5 values
dataset.isna().sum()

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset.isna().sum()

In [ ]:
dataset['Date']=pd.to_datetime(dataset[['Year', 'Month', 'Day', 'Hour']].rename(
                columns={'YY': 'year', 'MM': 'month', 'DD': 'day',
                         'hh': 'hour'}))
n_df=dataset.iloc[:,6:16]
#n_df.head()


Split the data into train and test
Now split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our model.

In [ ]:
n_df.shape
#n_df

In [ ]:
# let's split this data into train and test (80%, 20%)
train_dataset = n_df.iloc[0:round(n_df.shape[0]*.8),:]
test_dataset = n_df.drop(train_dataset.index)
test_dataset.shape

In [ ]:
# Now let's split the train_dataset into train and validation
train_X = train_dataset.iloc[0:round(train_dataset.shape[0]*.8),:]
valid_X = train_dataset.iloc[round(train_dataset.shape[0]*.8):,:]
test_y = test_dataset.iloc[0:round(test_dataset.shape[0]*.8),:]
#valid_y = 
test_y.shape


In [ ]:
train_dataset.head()
test_dataset.head()
train_dataset.shape
train_dataset.head()

colNames=list(train_dataset)
colNames

Look at the overall statistic

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("Electricity_KW_HR")
train_stats = train_stats.transpose()
train_stats


Split features from labels
Separate the target value, or "label", from the features. This label is the value that you will train the model to predict.

In [ ]:
tst_Elec=test_dataset['Electricity_KW_HR']
tst_Elec.head()

In [ ]:
train_labels = train_dataset.pop('Electricity_KW_HR')
test_labels = test_dataset.pop('Electricity_KW_HR')

In [ ]:
train_labels

In [ ]:
train_dataset

Normalize the data
Look again at the train_stats block above and note how different the ranges of each feature are.

It is good practice to normalize features that use different scales and ranges. Although the model might converge without feature normalization, it makes training more difficult, and it makes the resulting model dependent on the choice of units used in the input.

In [ ]:
train_stats['mean']

In [ ]:
#test_dataset

# this is z-score normalization

In [ ]:
# this is z-score normalization
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
#train_labels

# transformation

In [ ]:
# Order Normalize all data
def ordNorm():
    qt1=QuantileTransformer(n_quantiles=10, random_state=0)
    qt2=QuantileTransformer(n_quantiles=10, random_state=0)
    qt3=QuantileTransformer(n_quantiles=10, random_state=0)
    qt4=QuantileTransformer(n_quantiles=10, random_state=0)
    qt5=QuantileTransformer(n_quantiles=10, random_state=0)
    qt6=QuantileTransformer(n_quantiles=10, random_state=0)

    train_X=qt1.fit_transform(train_dataset) 
    test_X=qt2.fit_transform(test_dataset) 
    train_y=qt3.fit_transform(np.array(train_labels).reshape(-1,1))
    test_y=qt4.fit_transform(np.array(test_labels).reshape(-1,1))

    #X_t=qt5.fit_transform(train_dataset) 
    #y_t=qt6.fit_transform(train_labels)
    return train_X, train_y, test_X, test_y, qt4

# normalize using Yeo Johnson transformation
def yeoJohnson():   
    pt1=PowerTransformer()
    pt2=PowerTransformer()
    pt3=PowerTransformer()
    pt4=PowerTransformer()
    pt5=PowerTransformer()
    pt6=PowerTransformer()

    train_X=pt1.fit_transform(train_dataset) 
    test_X=pt2.fit_transform(test_dataset) 
    train_y=pt3.fit_transform(np.array(train_labels).reshape(-1,1))
    test_y=pt4.fit_transform(np.array(test_labels).reshape(-1,1))

    #X_t=pt5.fit_transform(features) 
    #y_t=pt6.fit_transform(labels.reshape(-1,1))
    return train_X, train_y, test_X, test_y, qt4


# normalize using the minMaxScaler #
def minMaxscaler():
    pt1=MinMaxScaler()
    pt2=MinMaxScaler()
    pt3=MinMaxScaler()
    pt4=MinMaxScaler()
    pt5=MinMaxScaler()
    pt6=MinMaxScaler()
    
    train_X=pt1.fit_transform(train_dataset) 
    test_X=pt2.fit_transform(test_dataset) 
    train_y=pt3.fit_transform(np.array(train_labels).reshape(-1,1))
    test_y=pt4.fit_transform(np.array(test_labels).reshape(-1,1))

    #X_t=pt5.fit_transform(features) 
    #y_t=pt6.fit_transform(labels.reshape(-1,1))
    return train_X, train_y, test_X, test_y, qt4

In [ ]:
#train_X, train_y, test_X, test_y, qt4=ordNorm()
#train_X, train_y, test_X, test_y, qt4=yeoJohnson()
train_X, train_y, test_X, test_y, qt4=minMaxscaler()

In [ ]:
plt.hist(df['Solar_Elevation'], density=True, bins=30)

In [ ]:
len(train_dataset.keys())

This normalized data is what we will use to train the model.

Build the model   
Let's build our model. Here, we'll use a Sequential model with two densely connected hidden layers, and an output layer that returns a single, continuous value. The model building steps are wrapped in a function, build_model, since we'll create a second model, later on.

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [ ]:
model = build_model()
model.summary() # Use the .summary method to print a simple description of the model

Now try out the model. Take a batch of 10 examples from the training data and call model.predict on it.

In [ ]:
#example_batch = train_dataset[:10]
#example_result = model.predict(example_batch)
#example_result

It seems to be working, and it produces a result of the expected shape and type.

Train the model
Train the model for 1000 epochs, and record the training and validation accuracy in the history object.

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 100

#history = model.fit(normed_train_data, train_labels, epochs=EPOCHS, validation_split = 0.2, verbose=0, callbacks=[PrintDot()])
history = model.fit(train_X, train_y, epochs=EPOCHS, validation_split = 0.2, verbose=0, callbacks=[PrintDot()])


Visualize the model's training progress using the stats stored in the history object.

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,0.2])
  plt.legend()
  
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,0.1])
  plt.legend()
  plt.show()


plot_history(history)

This graph shows little improvement, or even degradation in the validation error after about 100 epochs. Let's update the model.fit call to automatically stop training when the validation score doesn't improve. We'll use an EarlyStopping callback that tests a training condition for every epoch. If a set amount of epochs elapses without showing improvement, then automatically stop the training.

You can learn more about this callback here.

The graph shows that on the validation set, the average error is usually around +/- 2 MPG. Is this good? We'll leave that decision up to you.

Let's see how well the model generalizes by using the test set, which we did not use when training the model. This tells us how well we can expect the model to predict when we use it in the real world.

In [ ]:
#loss, mae, mse = model.evaluate(test_X, test_y, verbose=0)

#print("Testing set Mean Abs Error: {:5.2f}".format(mae))
#print("Testing set Mean Square Error: {:5.2f}".format(mse))

train_predictions = model.predict(train_X).flatten()
test_predictions = model.predict(test_X).flatten()
#print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(train_y, train_predictions)))
#print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(test_y, test_predictions)))


test_r2 = round(r2_score(test_y, test_predictions), 3)
train_r2 = round(r2_score(train_y, train_predictions), 3)
rmse = round(np.sqrt(mean_squared_error(test_y, test_predictions)),5)
rmse_trn = round(np.sqrt(mean_squared_error(train_y, train_predictions)),5)
mse = round(mean_squared_error(test_y, test_predictions), 5)
mse_trn = round(mean_squared_error(train_y, train_predictions), 5)
exp_var = round(explained_variance_score(test_y, test_predictions), 3)
exp_var_trn = round(explained_variance_score(train_y, train_predictions), 3)
mae= round(mean_absolute_error(test_predictions,test_y), 5)
mae_trn=round(mean_absolute_error(train_predictions,train_y), 5)

#explained_variance_score(train_labels, pred_trn)
print('Random Forest Regression Model Metrics:',
                  '\n\nTest R-squared:\t\t', test_r2,
                  '\nTrain R-squared:\t', train_r2,
                  '\nRMSE:\t\t\t', rmse,
                  '\nTrain RMSE:\t\t', rmse_trn,
                  '\nMSE:\t\t\t', mse,
                  '\nTrain MSE:\t\t', mse_trn,
                  '\nExplained Variance:\t', exp_var,
                  '\nTrain Explained Variance:', exp_var_trn,
                  '\nMAE:\t\t\t', mae,
                  '\nTrain MAE:\t\t', mae_trn)

#

# Make predictions

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values ')
plt.ylabel('Predictions ')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
test_predictions

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")

It's not quite gaussian, but we might expect that because the number of samples is very small.

Now let's build another model with different parament for number of hidden nodes. In this case 100 notes

In [ ]:
# Display training progress by printing a single dot for each completed epoch
#class PrintDot(keras.callbacks.Callback):
#  def on_epoch_end(self, epoch, logs):
#    if epoch % 100 == 0: print('')
#    print('.', end='')

#EPOCHS = 1000

#history = model1.fit(
#  normed_train_data, train_labels,
#  epochs=EPOCHS, validation_split = 0.2, verbose=0,
#  callbacks=[PrintDot()])

In [ ]:
def build_model1():
  model1 = keras.Sequential([
    layers.Dense(100, activation=tf.nn.tanh, input_shape=[len(train_dataset.keys())]),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model1.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model1

model1 = build_model1()
model1.summary()


# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model1.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

#plot_history(history)

############################
loss, mae, mse = model1.evaluate(normed_test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f}".format(mae))
print("Testing set Mean Square Error: {:5.2f}".format(mse))
train_predictions = model1.predict(normed_train_data).flatten()
test_predictions = model1.predict(normed_test_data).flatten()
print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(train_labels, train_predictions)))
print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(test_labels, test_predictions)))

In [ ]:
############### Order Normal train and Test Data #######################

model2 = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model2.fit(orderNorm_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)
###################################################

loss, mae, mse = model2.evaluate(orderNorm_test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f}".format(mae))
print("Testing set Mean Square Error: {:5.2f}".format(mse))

train_predictions = model2.predict(orderNorm_train_data).flatten()
test_predictions = model2.predict(orderNorm_test_data).flatten()
print("The orderNorm_train_data R2 score on the Train set is:\t{:0.3f}".format(r2_score(train_labels, train_predictions)))
print("The orderNorm_test_data R2 score on the Test set is:\t{:0.3f}".format(r2_score(test_labels, test_predictions)))

test_predictions = model2.predict(orderNorm_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values ')
plt.ylabel('Predictions ')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")

 # no normalized train and Test Data

In [ ]:
############### no normalized train and Test Data #######################

model3 = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model3.fit(train_dataset, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)
###################################################

loss, mae, mse = model3.evaluate(test_dataset, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f}".format(mae))
print("Testing set Mean Square Error: {:5.2f}".format(mse))

train_predictions = model3.predict(train_dataset).flatten()
test_predictions = model3.predict(test_dataset).flatten()
print("The train_dataset R2 score on the Train set is:\t{:0.3f}".format(r2_score(train_labels, train_predictions)))
print("The test_dataset R2 score on the Test set is:\t{:0.3f}".format(r2_score(test_labels, test_predictions)))

test_predictions = model3.predict(test_dataset).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values ')
plt.ylabel('Predictions ')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error")
_ = plt.ylabel("Count")